___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />

# Microestructuras y Sistema de Trading
## Laboratorio 1

### EQUIPO:
#### - Tere Castillo
#### - Ana Laura Correa
#### - Gaspar Benitez
#### - Marysol Cantarero

#### Situación: 
Acabas de entrar a trabajar a una operadora de fondos de inversión como analista financiero dentro de la mesa de análisis del mercado de renta variable. En la junta trimestral se plantea la posibilidad de crear un fondo propio con exposición al mercado de valores de USA tratando de replicar a los ETFs conocidos como ishares que la empresa BlackRock ofrece. 

La idea es replicar un ETF con posiciones en directo. Te han asignado la tarea de hacer un ejercicio simple de replicar la construcción de un portafolio de inversión y un criterio de rebalanceo lo más parecido a un ETF de ishares que tu selecciones. Esto con la finalidad de comparar los resultados de haber rebalanceado periódicamente el portafolio, ó, no haberlo hecho y continuar todo un año con las mismas posiciones.


#### Resolucion del problema

Los ETF(Exchange Trade Funds) son fondos o instrumentos de inversión cuyos títulos cotizan en la bolsa, al igual que una acción bursátil,pudiendose comprar y vender en cualquier momento, sin necesidad de esperar al cierre del mercado. Su objetivo es replicar el comportamiento de los activos que componen un índice bursátil, ya sea de renta fija, variable, divisas o materias primas u otros activos financieros.

Para este problema se analizara el ETF "iShares U.S. Financials ETF (IYF)" para el mercado de renta variable de USA con posiciones en directo tomando información del 01/Marzo/2018 al 01/Marzo/2019. Sin embargo, el código a realizar debe ser repetible para cualquier ETF.

Las 3 medidas de atribución al desempeño que se eligieron para la comparación de los resultados son: Rendimiento, volatilidad y medidas de desempeño.

#### Características mínimas de solución: 
Debido a tu formación, te piden que este proceso que construyas sea repetible para cualquier ETF y también incluir 3 medidas de atribución al desempeño, tanto para el caso que el ETF nunca hubiese sido rebalanceado como para los casos donde sí. 

En la mesa de análisis están interesados en conocer el proceso que sigues para adquirir, limpiar, dar formato y utilizar los datos, y a pesar que el equipo senior de análisis está compuesto por personal altamente calificado en temas de economía y finanzas, ellos no saben programar por lo que te piden que incluyas el código utilizado, y que sea de manera ordenada y explicando el sentido de las funciones y paqueterías que requeriste. 

Comparar con 1 año de información: 01/Marzo/2017 al 01/Marzo/2018 

Utilizar un ETF de Ishares para el mercado de renta varialbe de USA 

Entregar un Jupyter Notebook con explicación y comentarios en el código 

Crear y actualizar continuamente un repositorio en GitHub para el proyecto.

#### Resolucion del problema

Los ETF(Exchange Trade Funds) son fondos o instrumentos de inversión cuyos títulos cotizan en la bolsa, al igual que una acción bursátil,pudiendose comprar y vender en cualquier momento, sin necesidad de esperar al cierre del mercado. Su objetivo es replicar el comportamiento de los activos que componen un índice bursátil, ya sea de renta fija, variable, divisas o materias primas u otros activos financieros.

Para este problema se analizara el ETF "iShares U.S. Financials ETF (IYF)" para el mercado de renta variable de USA con posiciones en directo tomando información del 01/Marzo/2017 al 01/Marzo/2018. Sin embargo, el código a realizar debe ser repetible para cualquier ETF.

Las 3 medidas de atribución al desempeño que se eligieron para la comparación de los resultados son: Rendimiento, volatilidad y medidas de desempeño.

In [1115]:
# DESCARGA DE LIBRERIAS
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

### DESCARGA DE DATOS

In [1116]:
# FUNCIÓN PARA DESCARGA DE ARCHIVOS CON DATOS HOLDINGS ETF Y LOS TICKERS(INDIDES) DE CADA ARCHIVO (ISHARES)
def holdings_csv(narch,name):
    holdings = []
    tickers = []
    for i in range(narch):
        r = name+str(i)+".csv"
        info = pd.read_csv(r,header=10, index_col= 0)
        info = info.dropna() # quitando los datos vacios.
        info = info.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
        holdings.append(info)
        tickers.append(list(holdings[i].index))
    return holdings, tickers
narch = 13
holdings, tickers = holdings_csv(narch,"IYF_holdings")

##### Lo unico que se moverá son los pesos. price y shares quedan igual.
### CASO 1 : CONSTRUCCIÓN DE PORTAFOLIO DE INVERSIÓN SIN REBALANCEO (mismos precios).

In [932]:
# Encontrando los tickers diferentes y quitandolas para tener mismo índice(ajustando tickers)
for z in range(narch): 
    for i in range(narch-1):
        if len(tickers[z])>=len(tickers[i+1]):
            for j in tickers[z]:
                if j not in tickers[i+1]:
                    tickers[z].remove(j)
        else:
            for j in tickers[i+1]:
                if j not in tickers[z]:
                    tickers[i+1].remove(j)
indice = pd.DataFrame(tickers).T

In [1118]:
# Acomodo de información de acuerdo a Tickers (Weight(%), Price y Shares)
DtF = pd.DataFrame()
DtF1 = pd.DataFrame()
DtF2 =  pd.DataFrame()
for i in range(narch):
    l= []
    l1= []
    l2= []
    for j in range(indice.shape[0]):
        l.append(holdings[i]['Weight (%)'][indice[0][j]])
        l1.append(holdings[i]['Price'][indice[0][j]])
        l2.append(holdings[i]['Shares'][indice[0][j]])
    DtF['Holdings'+str(i)+''] = l
    DtF1['Holdings'+str(i)+''] = l1
    DtF2['Holdings'+str(i)+''] = l2
Weights = DtF.set_index(indice[0])
Prices = DtF1.set_index(indice[0])
Shares = DtF2.set_index(indice[0])

In [1119]:
ST = []
for j in range(narch):
    st = []
    for i in Shares['Holdings'+str(j)+'']:
        string = float(i.replace(',',''))
        st.append(string)
    ST.append(st)   
sh = pd.DataFrame(ST).T
sh = sh.set_index(indice[0])

##### No es el mismo valor que el etf original ya que se eliminaron tickers que no tenían todos los periodos

In [1150]:
#FUNCIÓN PARA CALCULAR VALOR DEL ETF
def v_etf(narch):
    v_etf2 = []
    for i in range(narch):
        H = 'Holdings'+str(i)+''
        st = []
        pr = []
        if type(Prices[H][0]) == np.float64:  
            for i in Shares[H]:
                string = float(i.replace(',',''))
                st.append(string) 
            a1 = Weights[H]*Prices[H]*st
            v_etf1 = sum(a1)
            v_etf2.append(v_etf1)
        else:
            for i in Shares[H]:
                string = float(i.replace(',',''))
                st.append(string)     
            for j in Prices[H]:
                prices = float(j.replace(',',''))
                pr.append(prices)  
            a1 = Weights[H]*pr*st
            v_etf1 = sum(a1)
            v_etf2.append(v_etf1)
    return(v_etf2)

In [1151]:
v_etf(13)

[4702939008.795299,
 3981849381.8813024,
 3667813409.073704,
 3475882852.553502,
 3911557212.773802,
 4010368973.189203,
 4167998917.0356,
 4367289647.971802,
 4745123731.519703,
 4711089644.51111,
 4545443525.726699,
 5353861485.607294,
 5628959832.798496]

In [1017]:
# FUNCIÓN PARA CALCULAR VALOR DEL ETF
def v_etf(narch):
    v_etf2 =[]
    for i in range(narch):
        w = holdings[i]['Weight (%)'] 
        p = holdings[i]['Price']   # precios
        s = holdings[i]['Shares']  #número de acciones 
        st = []
        pr = []
        if type(p[0]) == np.float64:
            for i in s:
                string = float(i.replace(',',''))
                st.append(string)
            a1 = w*p*st # Valor en dinero de cada acción
            v_etf1 = sum(a1) # Valor Total ETF
            v_etf2.append(v_etf1)
        else:
            for i in s:
                string = float(i.replace(',',''))
                st.append(string)
            for j in p:
                prices = float(j.replace(',',''))
                pr.append(prices)
            a1 = w*pr*st # Valor en dinero de cada acción
            v_etf1 = sum(a1) # Valor Total ETF
            v_etf2.append(v_etf1)    
    return (v_etf2)
v_etf = v_etf(narch)
v_etf # Valor Total ETF

[4710243437.943899,
 3988107610.9762025,
 3673748875.904403,
 3482149652.4258018,
 3918183392.8238025,
 4016746801.783402,
 4174797106.7400994,
 4374789295.864503,
 4752277165.981703,
 4717794817.270909,
 4551314846.0201,
 5359660229.257094,
 5634231631.184796]

In [ ]:
# FUNCIÓN PARA CALCULAR VALOR DEL ETF
def v_etf(narch):
    v_etf2 =[]
    for i in range(narch):
        w = holdings[i]['Weight (%)'] 
        p = holdings[i]['Price']   # precios
        s = holdings[i]['Shares']  #número de acciones 
        st = []
        pr = []
        if type(p[0]) == np.float64:
            for i in s:
                string = float(i.replace(',',''))
                st.append(string)
            a1 = w*p*st # Valor en dinero de cada acción
            v_etf1 = sum(a1) # Valor Total ETF
            v_etf2.append(v_etf1)
        else:
            for i in s:
                string = float(i.replace(',',''))
                st.append(string)
            for j in p:
                prices = float(j.replace(',',''))
                pr.append(prices)
            a1 = w*pr*st # Valor en dinero de cada acción
            v_etf1 = sum(a1) # Valor Total ETF
            v_etf2.append(v_etf1)    
    return (v_etf2)
v_etf = v_etf(narch)
v_etf # Valor Total ETF

In [100]:
# para esto se necesitan ajustar los tickers para que sean iguales.
w = holdings[0]['Weight (%)'] # mismos pesos
v_etf2 = []
for i in range(narch):
    p = holdings[i]['Price']   # precios
    s = holdings[i]['Shares']  #número de acciones
    st = []
    pr = []
    if type(p[0]) == np.float64:
        for i in s:
            string = float(i.replace(',',''))
            st.append(string)
        a1 = w*p*st # Valor en dinero de cada acción
        v_etf1 = sum(a1) # Valor Total ETF
        v_etf2.append(v_etf1)
    else:
        for i in s:
            string = float(i.replace(',',''))
            st.append(string)
        for j in p:
            prices = float(j.replace(',',''))
            pr.append(prices)
        a1 = w*pr*st # Valor en dinero de cada acción
        v_etf1 = sum(a1) # Valor Total ETF
        v_etf2.append(v_etf1)

ValueError: operands could not be broadcast together with shapes (289,) (287,) 

In [ ]:
#a = p*w*s # Valor en dinero de cada acción
#v_etf_total = sum(a) #Valor del portafolio o etf, suma de los valores de las acciones

In [22]:
# Desviación estandar
nav = pd.read_excel('iShares-US-Financials-ETF_fund.xls',sheet_name=1, index_col= 0)
nav = nav["NAV per Share"][378:630]
#nav = list(nav)
#nav.reverse()
desv1 = np.std(nav)
mean1 = np.mean(nav)

In [23]:
nav

As Of
Feb 14, 2018    120.756139
Feb 13, 2018    118.639644
Feb 12, 2018    118.088112
Feb 09, 2018    116.745077
Feb 08, 2018    114.493401
                   ...    
Feb 22, 2017    106.962131
Feb 21, 2017    106.977532
Feb 17, 2017    106.314475
Feb 16, 2017    106.299564
Feb 15, 2017    106.290030
Name: NAV per Share, Length: 252, dtype: float64

### CRITERIO DE RE BALANCEO.

In [ ]:
#SERIE DE TIEMPO HACIENDO RE BALANCEO
plt.title("Serie de tiempo ETF IYF")
plt.ylabel("NAV")
plt.xlabel("Fechas")
plt.axvline(x=0, c = "green")
plt.axvline(x=22, c = "greenyellow")
plt.axvline(x=43, c = "tomato")
plt.axvline(x=64, c = "lightcoral")
plt.axvline(x=85, c = "aqua")
plt.axvline(x=106, c = "plum")
plt.axvline(x=125, c = "mediumvioletred")
plt.axvline(x=147, c = "lawngreen")
plt.axvline(x=166, c = "greenyellow")
plt.axvline(x=186, c = "gold")
plt.axvline(x=209, c = "teal")
plt.axvline(x=229, c = "lavender")
plt.axvline(x=251, c = "darkmagenta")
plt.plot(nav, c = "k")

### COMPARACIÓN DE RESULTADOS: REBALANCEO VS. MISMAS POSICIONES

In [ ]:
# MEDIDAS DE ATRIBUCIÓN AL DESEMPEÑO

### CONCLUSIONES
.....

In [ ]:
quandl.ApiConfig.api_key = "My9ygJsz7yTDx_9jcneH"  # API key Quandl
fechas = list(['2017-02-28', '2017-03-31', '2017-04-28', '2017-05-31', '2017-06-30',
                 '2017-07-31', '2017-08-31', '2017-09-29', '2017-10-31', '2017-11-30',
                 '2017-12-29', '2018-01-31', '2018-02-28'])

In [ ]:
# FUNCIÓN PARA DESCARGA DE DATOS HISTÓRICOS (QUANDL)
def download_quandl(fechas, narch):
    d2 = []  # Empty dictionary
    for i in range(narch):  # Loop to Download data from Quandl
        index= holdings[i].index
        q, h = holdings[i].shape
        ticker_r = list(index)  # ticker = 'WIKI/AAPL'
        date = {'gte': fechas[i], 'lte': fechas[i] }
        column = {'columns': ['ticker', 'date', 'Adj_close']}
        data = quandl.get_table('WIKI/PRICES', qopts=column, ticker=ticker_r, date=date)
        data = data.set_index('ticker')
        d2.append(data)
    return d2
prices = download_quandl(fechas,narch)

In [ ]:
# LIMPIEZA DE DATOS 
def clean_data(narch, prices, holdings):
    d =[]
    hold = []
    for i in range(narch):
        a = set(list(prices[i].index))
        b = set(list(holdings[i].index))
        c = []
        for item in b:
            if item not in a:
                 c.append(item)
        d.append(c)
        holdi = holdings[i].drop(d[i]) # Se eliminan datos de ishares que no estan en quandl.
        hold.append(holdi)
    return hold
d = clean_data(narch, prices, holdings)
d

### BIBLIOGRAFÍA
https://www.portafolio.co/economia/finanzas/son-etf-293754

https://economipedia.com/definiciones/etf-fondos-cotizados.html

https://www.ishares.com/us/products/239508/#/

https://www.quandl.com/tools/python

